In [2]:
import pandas as pd
#from pandas import option_context
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import wordnet as wn
from nltk.probability import FreqDist
from tqdm import tqdm
import re
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
import spacy

# Increase limit of characters processed
# Need to specify to not use spcifics models to avoid overload
# nlp_es.max_length = 2000000


## (fonctions)

In [75]:
# funct. import text, clean withite spaces, return cleaned text as list
def clean_input(txt_file):
    file_content = open(txt_path + file + ".txt", "r").readlines()
    file_cleaned = []
    for line in file_content:
        if not line.isspace():
            file_cleaned.append(line.strip('\n'))
    file_cleaned = '-'.join(file_cleaned)
    return(file_cleaned)


def remove_html_tags(text):
    clean = re.sub('<.*>','',text)
    return(clean)

## Textual corpus


In [76]:
# TODO : instead of length os text, transform into number of tokens and add csv
# Create dictionary to store content
corpus = {}
# Define files names
# TODO get from directory
name_files_txt = ['llb_new','llb_old','t_ohcr','s_ohcr','t_cru']
# explicit path
# TODO clean name
txt_path = 'sources/txt/'
# Fill the doctionary with the clean content
for file in name_files_txt:
    corpus[file] = clean_input(file)

In [77]:
directory = 'sources/txt/spa_ebible'
spa_ebible = {}
master_f = []

for filename in os.listdir(directory):
    f = os.path.join(directory,filename)
    if os.path.isfile(f):
        if filename.endswith(".txt"):
            rename = re.search(r'([0-9]{3,3}.*[0-9]{2,2})', filename)
            content = open(os.path.join(directory, filename), "rt",encoding="UTF-8-sig").read()
            content = re.sub('\n','',content)  
            master_f.append(content)          
            spa_ebible[rename.group(0)] = content

spa_ebible =  dict(sorted(spa_ebible.items()))

directory = 'sources/txt/tik_ebible'
tik_ebible = {}
master_f = []

for filename in os.listdir(directory):
    f = os.path.join(directory,filename)
    if os.path.isfile(f):
        if filename.endswith(".txt"):
            rename = re.search(r'([0-9]{3,3}.*[0-9]{2,2})', filename)
            content = open(os.path.join(directory, filename), "rt",encoding="UTF-8-sig").read()
            content = re.sub('\n','',content)
            master_f.append(content)          
            tik_ebible[rename.group(0)] = content

tik_ebible =  dict(sorted(tik_ebible.items()))

### HTML files, needs cleaning

# directory = 'sources/original/tcaNT_html'
# tik_ebible_full = {}

# for filename in os.listdir(directory):
#     f = os.path.join(directory,filename)
#     if os.path.isfile(f):
#         if filename.endswith(".htm"):
#             output = pypandoc.convert_file(f, 'rst',format = 'html')
#             filename = re.sub('\.htm','',filename)
#             tik_ebible_full[filename] = output

# directory = 'sources/original/spavbl_html'
# spa_ebible_full = {}

# #spa_ebible_full
# for key, value in spa_ebible_full.items():
#     # Clean content by removing all tags and content from inside.
#     # TODO : Keep hetml descritpion tags as meta data

#     spa_ebible_full[key] = remove_html_tags(value)
# # Sort dictionary to sort by apostle

# spa_ebible_full =  dict(sorted(spa_ebible_full.items()))

# for key, value in tik_ebible_full.items():
#     tik_ebible_full[key] = remove_html_tags(value)
#     tik_ebible_full =  dict(sorted(tik_ebible_full.items()))


# def clean_annotate_ebible(chapter_txt):
#     #char_to_remove = ['\-','\`','``','\.','\s*\>\s*','\s*<\s*','copyright','© 2008 WBT','_']
#     container_type = ['mt',',main','m','q','p','s','chapterlabel']
#     # from container :: [a-z]* to .., container annotation.
#     c_chapter = re.sub('\.\. container\:\:','',chapter_txt)
#     c_chapter = re.sub('\n','',c_chapter)
#     #c_chapter = re.search('^(.*?)[a-z]*','',chapter_txt)
#     #c_chapter = re.search('(\w+)',chapter_txt)
#     #print(c_chapter.group(0))
#     return(c_chapter)


In [78]:
def align_content_bible(dic1,dic2):
    bil_dict = {}
    common_chapters = [i for i in dic1.keys() if i in dic2.keys()]
    for element in common_chapters:
        bil_dict[element] = {'spanish':'','tikuna':''}
        bil_dict[element]['tikuna'] = dic1[element]
        bil_dict[element]['spanish'] = dic2[element]
    return(bil_dict)
bil_dict = align_content_bible(tik_ebible,spa_ebible)
# Remove first entry describing the corpus
del bil_dict['000_000_000']

In [240]:
bil_dict[1] = {'spanish':'','tikuna':''}
bil_dict[1]['tikuna'] = corpus['t_ohcr']
bil_dict[1]['spanish'] = corpus['s_ohcr']


In [215]:
ts_dic = pd.read_csv(r'../dic_preprocessing/output.csv')
ts_dic = ts_dic[ts_dic['example_spanish'].notna()].reset_index(drop=True)
ex_t = ts_dic['example_tikuna']
ex_s = ts_dic['example_spanish']
# Keep the index of the source df ?
def align_examples(tikuna,spanish):
    examplen = {}
    for i in range(len(tikuna)):
        examplen[i] = {'spanish':'','tikuna':''}
        examplen[i]['tikuna'] = tikuna[i]
        examplen[i]['spanish'] = spanish[i]
    return(examplen)
dic_examples = align_examples(ex_t,ex_s)


In [ ]:
master_output_training = {'ebible':'','ohcr':'','examples':''}
master_output_training['ebible'] = bil_dict
master_output_training['ohcr'] = bil_dict['ohcr']
master_output_training['examples'] = dic_examples

# Master Content Database 

The data is store in a dictionnary here is an small overview of the data

| Key    | Subkeys      |   Value   | Content   ||
| :---   |    :----:    |   ---:    |---:       |--:|
|TS_DIC  | Column names |   Tokens  | translations||
|LLB_NEW | Chapter Name |   Tikuna  |||
|LLB_OLD | Chapter Name |   Tikuna  |||
|T_Ebible| Chapter Name |   Tikuna  |||
|S_Ebible| Chapter Name |   Spanish |||
|T_OHCR  | Alinea       |   Tikuna  |||
|S_OHCR  | Alinea       |   Spanish |||
|T_CRU   | None         |   Tikuna  |||

*TODO : Draw the data structure tree. What has to be kept in hierachical order, what part pas whole text ?*
*Do both for modularity*



